In [1]:
# import libraries
import numpy as np
import pandas as pd
import pickle as pkl

import torch
import torch.nn as nn
from torch.nn.functional import one_hot 
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch.optim as optim

from TorchCRF import CRF
import tqdm
import blosum as bl

SAVE = True

In [2]:
data = pd.read_excel("./Data/peptides10.xlsx")

/Users/lanhongyi/opt/anaconda3/envs/dsl_project/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [3]:
data.head()

,Unnamed: 0,MMP1,MMP2,MMP3,MMP7,MMP8,MMP9,MMP10,MMP11,MMP12,MMP13,MMP14,MMP15,MMP16,MMP17,MMP19,MMP20,MMP24,MMP25,construct
0,GASGGMGNKG,-0.59,-0.80,-1.00,-1.35,-1.39,-1.14,-0.97,-0.87,-1.37,-1.27,-0.26,-0.60,-0.84,-0.74,-1.55,-0.72,-1.14,0.07,BF015
1,TKIYNYDCEP,-1.08,-1.11,-0.37,-1.07,-0.81,-0.52,-0.58,-0.77,-0.41,-0.59,-0.93,-1.00,-1.40,0.57,0.28,0.45,-0.64,-0.19,BF016
2,TKIYDYDCEP,-0.93,-0.39,0.17,-0.42,-0.83,-0.02,-0.39,-0.58,-0.02,-0.16,-0.76,-0.57,-0.82,0.95,0.96,0.24,-0.66,1.15,BF018
3,TKIYDLDCEP,-1.28,-1.29,-0.66,-0.16,-0.41,-1.04,-0.55,-0.34,-0.23,-0.58,-0.41,-0.76,-1.70,0.33,-0.98,-0.44,-1.09,-0.17,BF019
4,GASGGLGNKG,-1.28,-1.00,-1.42,-1.64,-0.16,-1.07,-0.93,-1.19,-1.71,-1.35,-0.46,-1.25,-1.12,-0.31,-0.29,-0.70,-0.95,0.30,BF024


In [4]:
data.iloc[:, 0] = data.iloc[:, 0].map(lambda x: x[1:-1])

In [6]:
data = data.iloc[:, :-1]

In [7]:
data.head()

,Unnamed: 0,MMP1,MMP2,MMP3,MMP7,MMP8,MMP9,MMP10,MMP11,MMP12,MMP13,MMP14,MMP15,MMP16,MMP17,MMP19,MMP20,MMP24,MMP25
0,ASGGMGNK,-0.59,-0.80,-1.00,-1.35,-1.39,-1.14,-0.97,-0.87,-1.37,-1.27,-0.26,-0.60,-0.84,-0.74,-1.55,-0.72,-1.14,0.07
1,KIYNYDCE,-1.08,-1.11,-0.37,-1.07,-0.81,-0.52,-0.58,-0.77,-0.41,-0.59,-0.93,-1.00,-1.40,0.57,0.28,0.45,-0.64,-0.19
2,KIYDYDCE,-0.93,-0.39,0.17,-0.42,-0.83,-0.02,-0.39,-0.58,-0.02,-0.16,-0.76,-0.57,-0.82,0.95,0.96,0.24,-0.66,1.15
3,KIYDLDCE,-1.28,-1.29,-0.66,-0.16,-0.41,-1.04,-0.55,-0.34,-0.23,-0.58,-0.41,-0.76,-1.70,0.33,-0.98,-0.44,-1.09,-0.17
4,ASGGLGNK,-1.28,-1.00,-1.42,-1.64,-0.16,-1.07,-0.93,-1.19,-1.71,-1.35,-0.46,-1.25,-1.12,-0.31,-0.29,-0.70,-0.95,0.30
